In [1]:
from brainlit.utils.session import NeuroglancerSession
from brainlit.utils.swc import graph_to_paths
from brainlit.viz.visualize import napari_viewer

import numpy as np
import h5py
import os
from skimage import io
from pathlib import Path

D:\Anaconda3\envs\brainlit\lib\site-packages\napari\__init__.py:44: UserWarning: 
    napari was tested with QT library `>=5.12.3`.
    The version installed is 5.9.6. Please report any issues with this
    specific QT version at https://github.com/Napari/napari/issues.
    
  warn(message=warn_message)


In [2]:
dir = "s3://open-neurodata/brainlit/brain1"
dir_segments = "s3://open-neurodata/brainlit/brain1_segments"
mip = 0
v_id = 0
radius = 75

# get image and center point
ngl_sess = NeuroglancerSession(mip = mip, url = dir, url_segments = dir_segments)
img, bbox, vox = ngl_sess.pull_voxel(2, v_id, radius)
print(f"\n\nDownloaded volume is of shape {img.shape}, with total intensity {sum(sum(sum(img)))}.")

Downloading: 100%|█████████████████████████████████████████████████████████████████████| 24/24 [00:02<00:00, 10.76it/s]



Downloaded volume is of shape (151, 151, 151), with total intensity 4946609.


In [3]:
save_path = "C:/Users/Tanch/ffn/brainlit/images"
test_path = save_path + "/image1.h5"
with h5py.File(test_path, 'w') as f:
  f.create_dataset('raw', data = img, compression = 'gzip') 

In [4]:
base_dir = Path("D:/Study/Nuero Data Design/brainlit")
data_dir = base_dir / "benchmarking_datasets"
im_dir = data_dir / "Images"
mask_dir = base_dir / "benchmarking_masks"
gfp_files = list(im_dir.glob("**/*.tif"))

im_num = 0
im_path = gfp_files[im_num]
print("The image used for training:")
print(im_path)
im = io.imread(im_path, plugin="tifffile")
im = np.swapaxes(im, 0, 2)
file_name = im_path.parts[-1][:-8]
mask_file = file_name + "_mask.tif"
mask_path = mask_dir / mask_file
mask = io.imread(mask_path, plugin="tifffile")
# The list of allowed values: 
#     bfloat16, float16, float32, float64, uint8, int8, int16, int32, int64, 
#     complex64, quint8, qint8, qint32, string, bool, complex128
im = im.astype("int16")
mask = mask.astype("int32")

The image used for training:
D:\Study\Nuero Data Design\brainlit\benchmarking_datasets\Images\test_1-gfp.tif


In [5]:
napari_viewer(im, labels=mask, label_name="mask")

In [6]:
train_dir = save_path + "/" + file_name
if not os.path.exists(train_dir):
    os.makedirs(train_dir)
train_path = train_dir + "/grayscale_maps.h5"
with h5py.File(train_path, 'w') as f:
  f.create_dataset('raw', data = im, compression = 'gzip') 
mask_path = train_dir + "/groundtruth.h5"
with h5py.File(mask_path, 'w') as f:
  f.create_dataset('stack', data = mask, compression = 'gzip') 

In [7]:
im.shape

(330, 330, 100)

Preprocess the data (run the following scripts in cmd.exe)

Script 1:

    python compute_partitions.py \
        --input_volume brainlit/images/test_1/groundtruth.h5:stack \
        --output_volume brainlit/images/test_1/af.h5:af \
        --thresholds 0.025,0.05,0.075,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9 \
        --lom_radius 24,24,24 \
        --min_size 10000
        
Script 2:

    python build_coordinates.py \
         --partition_volumes validation1:brainlit/images/test_1/af.h5:af \
         --coordinate_output brainlit/images/test_1/tf_record_file \
         --margin 24,24,24

Inputs and outputs of Script 1 in cmd.exe (ignored the warnings):

    python compute_partitions.py --input_volume brainlit/images/test_1/groundtruth.h5:stack --output_volume brainlit/images/test_1/af.h5:af --thresholds 0.025,0.05,0.075,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9 --lom_radius 24,24,24 --min_size 10000

    I1108 19:54:07.485412 28424 compute_partitions.py:192] Done processing 11706
    I1108 19:54:07.830472 28424 compute_partitions.py:192] Done processing 11707
    I1108 19:54:08.183553 28424 compute_partitions.py:192] Done processing 11708
    I1108 19:54:08.530618 28424 compute_partitions.py:192] Done processing 11709
    I1108 19:54:08.871706 28424 compute_partitions.py:192] Done processing 11710
    I1108 19:54:09.232741 28424 compute_partitions.py:192] Done processing 11711
    I1108 19:54:09.577818 28424 compute_partitions.py:192] Done processing 11712
    I1108 19:54:09.931879 28424 compute_partitions.py:192] Done processing 11713
    I1108 19:54:10.281924 28424 compute_partitions.py:192] Done processing 11714
    I1108 19:54:10.638984 28424 compute_partitions.py:192] Done processing 11715
    I1108 19:54:10.989048 28424 compute_partitions.py:192] Done processing 11716
    I1108 19:54:11.333128 28424 compute_partitions.py:192] Done processing 11717
    I1108 19:54:11.683193 28424 compute_partitions.py:192] Done processing 11718
    I1108 19:54:12.036248 28424 compute_partitions.py:192] Done processing 11719
    I1108 19:54:12.377298 28424 compute_partitions.py:192] Done processing 11720
    I1108 19:54:12.732388 28424 compute_partitions.py:192] Done processing 11721
    I1108 19:54:13.079461 28424 compute_partitions.py:192] Done processing 11722
    I1108 19:54:13.414565 28424 compute_partitions.py:192] Done processing 11723
    I1108 19:54:13.768580 28424 compute_partitions.py:192] Done processing 11724
    I1108 19:54:14.124667 28424 compute_partitions.py:192] Done processing 11725
    I1108 19:54:14.465716 28424 compute_partitions.py:192] Done processing 11726
    I1108 19:54:14.812829 28424 compute_partitions.py:192] Done processing 11727
    I1108 19:54:15.164890 28424 compute_partitions.py:192] Done processing 11728
    I1108 19:54:15.518944 28424 compute_partitions.py:192] Done processing 11729
    I1108 19:54:15.864019 28424 compute_partitions.py:192] Done processing 11730
    I1108 19:54:16.222062 28424 compute_partitions.py:192] Done processing 11731
    I1108 19:54:16.568099 28424 compute_partitions.py:192] Done processing 11732
    I1108 19:54:16.914213 28424 compute_partitions.py:192] Done processing 11733
    I1108 19:54:17.264279 28424 compute_partitions.py:192] Done processing 11734
    I1108 19:54:17.613344 28424 compute_partitions.py:192] Done processing 11735
    I1108 19:54:17.957386 28424 compute_partitions.py:192] Done processing 11736
    I1108 19:54:18.305494 28424 compute_partitions.py:192] Done processing 11737
    I1108 19:54:18.649556 28424 compute_partitions.py:192] Done processing 11738
    I1108 19:54:18.995649 28424 compute_partitions.py:192] Done processing 11739
    I1108 19:54:19.350707 28424 compute_partitions.py:192] Done processing 11740
    I1108 19:54:19.705759 28424 compute_partitions.py:192] Done processing 11741
    I1108 19:54:20.051789 28424 compute_partitions.py:192] Done processing 11742
    I1108 19:54:20.400883 28424 compute_partitions.py:192] Done processing 11743
    I1108 19:54:20.747967 28424 compute_partitions.py:192] Done processing 11744
    I1108 19:54:21.101026 28424 compute_partitions.py:192] Done processing 11745
    I1108 19:54:21.437130 28424 compute_partitions.py:192] Done processing 11746
    I1108 19:54:21.784202 28424 compute_partitions.py:192] Done processing 11747
    I1108 19:54:22.134263 28424 compute_partitions.py:192] Done processing 11748
    I1108 19:54:22.482333 28424 compute_partitions.py:192] Done processing 11749
    I1108 19:54:22.830401 28424 compute_partitions.py:192] Done processing 11750
    I1108 19:54:23.180465 28424 compute_partitions.py:192] Done processing 11751
    I1108 19:54:23.529536 28424 compute_partitions.py:192] Done processing 11752
    I1108 19:54:23.874612 28424 compute_partitions.py:192] Done processing 11753
    I1108 19:54:24.229664 28424 compute_partitions.py:192] Done processing 11754
    I1108 19:54:24.588704 28424 compute_partitions.py:192] Done processing 11755
    I1108 19:54:24.934777 28424 compute_partitions.py:192] Done processing 11756
    I1108 19:54:25.288831 28424 compute_partitions.py:192] Done processing 11757
    I1108 19:54:25.642886 28424 compute_partitions.py:192] Done processing 11758
    I1108 19:54:25.996939 28424 compute_partitions.py:192] Done processing 11759
    I1108 19:54:26.336035 28424 compute_partitions.py:192] Done processing 11760
    I1108 19:54:26.684083 28424 compute_partitions.py:192] Done processing 11761
    I1108 19:54:27.031162 28424 compute_partitions.py:192] Done processing 11762
    I1108 19:54:27.367277 28424 compute_partitions.py:192] Done processing 11763
    I1108 19:54:27.712354 28424 compute_partitions.py:192] Done processing 11764
    I1108 19:54:28.056435 28424 compute_partitions.py:192] Done processing 11765
    I1108 19:54:28.393541 28424 compute_partitions.py:192] Done processing 11766
    I1108 19:54:28.744577 28424 compute_partitions.py:192] Done processing 11767
    I1108 19:54:29.099648 28424 compute_partitions.py:192] Done processing 11768
    I1108 19:54:29.447717 28424 compute_partitions.py:192] Done processing 11769
    I1108 19:54:29.803765 28424 compute_partitions.py:192] Done processing 11770
    I1108 19:54:30.154787 28424 compute_partitions.py:192] Done processing 11771
    I1108 19:54:30.495877 28424 compute_partitions.py:192] Done processing 11772
    I1108 19:54:30.850954 28424 compute_partitions.py:192] Done processing 11773
    I1108 19:54:31.205023 28424 compute_partitions.py:192] Done processing 11774
    I1108 19:54:31.564063 28424 compute_partitions.py:192] Done processing 11775
    I1108 19:54:31.913109 28424 compute_partitions.py:192] Done processing 11776
    I1108 19:54:32.263194 28424 compute_partitions.py:192] Done processing 11777
    I1108 19:54:32.611262 28424 compute_partitions.py:192] Done processing 11778
    I1108 19:54:32.957325 28424 compute_partitions.py:192] Done processing 11779
    I1108 19:54:33.297429 28424 compute_partitions.py:192] Done processing 11780
    I1108 19:54:33.656469 28424 compute_partitions.py:192] Done processing 11781
    I1108 19:54:34.003543 28424 compute_partitions.py:192] Done processing 11782
    I1108 19:54:34.346623 28424 compute_partitions.py:192] Done processing 11783
    I1108 19:54:34.704667 28424 compute_partitions.py:192] Done processing 11784
    I1108 19:54:35.047753 28424 compute_partitions.py:192] Done processing 11785
    I1108 19:54:35.387830 28424 compute_partitions.py:192] Done processing 11786
    I1108 19:54:35.737867 28424 compute_partitions.py:192] Done processing 11787
    I1108 19:54:36.092957 28424 compute_partitions.py:192] Done processing 11788
    I1108 19:54:36.424073 28424 compute_partitions.py:192] Done processing 11789
    I1108 19:54:36.773139 28424 compute_partitions.py:192] Done processing 11790
    I1108 19:54:37.130185 28424 compute_partitions.py:192] Done processing 11791
    I1108 19:54:37.465290 28424 compute_partitions.py:192] Done processing 11792
    I1108 19:54:37.812324 28424 compute_partitions.py:192] Done processing 11793
    I1108 19:54:38.176389 28424 compute_partitions.py:192] Done processing 11794
    I1108 19:54:38.526456 28424 compute_partitions.py:192] Done processing 11795
    I1108 19:54:38.876481 28424 compute_partitions.py:192] Done processing 11796
    I1108 19:54:39.236558 28424 compute_partitions.py:192] Done processing 11797
    I1108 19:54:39.585628 28424 compute_partitions.py:192] Done processing 11798
    I1108 19:54:39.944656 28424 compute_partitions.py:192] Done processing 11799
    I1108 19:54:40.297728 28424 compute_partitions.py:192] Done processing 11800
    I1108 19:54:40.640790 28424 compute_partitions.py:192] Done processing 11801
    I1108 19:54:40.997836 28424 compute_partitions.py:192] Done processing 11802
    I1108 19:54:41.350908 28424 compute_partitions.py:192] Done processing 11803
    I1108 19:54:41.703943 28424 compute_partitions.py:192] Done processing 11804
    I1108 19:54:42.049998 28424 compute_partitions.py:192] Done processing 11805
    I1108 19:54:42.389131 28424 compute_partitions.py:192] Done processing 11806
    I1108 19:54:42.743184 28424 compute_partitions.py:192] Done processing 11807
    I1108 19:54:43.102226 28424 compute_partitions.py:192] Done processing 11808
    I1108 19:54:43.436335 28424 compute_partitions.py:192] Done processing 11809
    I1108 19:54:43.788372 28424 compute_partitions.py:192] Done processing 11810
    I1108 19:54:44.151402 28424 compute_partitions.py:192] Done processing 11811
    I1108 19:54:44.490477 28424 compute_partitions.py:192] Done processing 11812
    I1108 19:54:44.849543 28424 compute_partitions.py:192] Done processing 11813
    I1108 19:54:45.200578 28424 compute_partitions.py:192] Done processing 11814
    I1108 19:54:45.574604 28424 compute_partitions.py:192] Done processing 11815
    I1108 19:54:45.921677 28424 compute_partitions.py:192] Done processing 11816
    I1108 19:54:46.274708 28424 compute_partitions.py:192] Done processing 11817
    I1108 19:54:46.634784 28424 compute_partitions.py:192] Done processing 11818
    I1108 19:54:46.990833 28424 compute_partitions.py:192] Done processing 11819
    I1108 19:54:47.357851 28424 compute_partitions.py:192] Done processing 11820
    I1108 19:54:47.703927 28424 compute_partitions.py:192] Done processing 11821
    I1108 19:54:48.064924 28424 compute_partitions.py:192] Done processing 11822
    I1108 19:54:48.403059 28424 compute_partitions.py:192] Done processing 11823
    I1108 19:54:48.751130 28424 compute_partitions.py:192] Done processing 11824
    I1108 19:54:49.117153 28424 compute_partitions.py:192] Done processing 11825
    I1108 19:54:49.463186 28424 compute_partitions.py:192] Done processing 11826
    I1108 19:54:49.822266 28424 compute_partitions.py:192] Done processing 11827
    I1108 19:54:50.172331 28424 compute_partitions.py:192] Done processing 11828
    I1108 19:54:50.515401 28424 compute_partitions.py:192] Done processing 11829
    I1108 19:54:50.868457 28424 compute_partitions.py:192] Done processing 11830
    I1108 19:54:51.223523 28424 compute_partitions.py:192] Done processing 11831
    I1108 19:54:51.570593 28424 compute_partitions.py:192] Done processing 11832
    I1108 19:54:51.930631 28424 compute_partitions.py:192] Done processing 11833
    I1108 19:54:52.288660 28424 compute_partitions.py:192] Done processing 11834
    I1108 19:54:52.639736 28424 compute_partitions.py:192] Done processing 11835
    I1108 19:54:53.005739 28424 compute_partitions.py:192] Done processing 11836
    I1108 19:54:53.345838 28424 compute_partitions.py:192] Done processing 11837
    I1108 19:54:53.696912 28424 compute_partitions.py:192] Done processing 11838
    I1108 19:54:54.046976 28424 compute_partitions.py:192] Done processing 11839
    I1108 19:54:54.393050 28424 compute_partitions.py:192] Done processing 11840
    I1108 19:54:54.744112 28424 compute_partitions.py:192] Done processing 11841
    I1108 19:54:55.100161 28424 compute_partitions.py:192] Done processing 11842
    I1108 19:54:55.447234 28424 compute_partitions.py:192] Done processing 11843
    I1108 19:54:55.792313 28424 compute_partitions.py:192] Done processing 11844
    I1108 19:54:56.160288 28424 compute_partitions.py:192] Done processing 11845
    I1108 19:54:56.509395 28424 compute_partitions.py:192] Done processing 11846
    I1108 19:54:56.855469 28424 compute_partitions.py:192] Done processing 11847
    I1108 19:54:57.209525 28424 compute_partitions.py:192] Done processing 11848
    I1108 19:54:57.550615 28424 compute_partitions.py:192] Done processing 11849
    I1108 19:54:57.893682 28424 compute_partitions.py:192] Done processing 11850
    I1108 19:54:58.255728 28424 compute_partitions.py:192] Done processing 11851
    I1108 19:54:58.609781 28424 compute_partitions.py:192] Done processing 11852
    I1108 19:54:58.953823 28424 compute_partitions.py:192] Done processing 11853
    I1108 19:54:59.304924 28424 compute_partitions.py:192] Done processing 11854
    I1108 19:54:59.650987 28424 compute_partitions.py:192] Done processing 11855
    I1108 19:54:59.996037 28424 compute_partitions.py:192] Done processing 11856
    I1108 19:55:00.339159 28424 compute_partitions.py:192] Done processing 11857
    I1108 19:55:00.696166 28424 compute_partitions.py:192] Done processing 11858
    I1108 19:55:01.041283 28424 compute_partitions.py:192] Done processing 11859
    I1108 19:55:01.382352 28424 compute_partitions.py:192] Done processing 11860
    I1108 19:55:01.740377 28424 compute_partitions.py:192] Done processing 11861
    I1108 19:55:02.095468 28424 compute_partitions.py:192] Done processing 11862
    I1108 19:55:02.431569 28424 compute_partitions.py:192] Done processing 11863
    I1108 19:55:02.775648 28424 compute_partitions.py:192] Done processing 11864
    I1108 19:55:03.134688 28424 compute_partitions.py:192] Done processing 11865
    I1108 19:55:03.469793 28424 compute_partitions.py:192] Done processing 11866
    I1108 19:55:03.818853 28424 compute_partitions.py:192] Done processing 11867
    I1108 19:55:04.198844 28424 compute_partitions.py:192] Done processing 11868
    I1108 19:55:04.545916 28424 compute_partitions.py:192] Done processing 11869
    I1108 19:55:04.889000 28424 compute_partitions.py:192] Done processing 11870
    I1108 19:55:05.249995 28424 compute_partitions.py:192] Done processing 11871
    I1108 19:55:05.596110 28424 compute_partitions.py:192] Done processing 11872
    I1108 19:55:05.937200 28424 compute_partitions.py:192] Done processing 11873
    I1108 19:55:06.296240 28424 compute_partitions.py:192] Done processing 11874
    I1108 19:55:06.650292 28424 compute_partitions.py:192] Done processing 11875
    I1108 19:55:06.998362 28424 compute_partitions.py:192] Done processing 11876
    I1108 19:55:07.344438 28424 compute_partitions.py:192] Done processing 11877
    I1108 19:55:07.690513 28424 compute_partitions.py:192] Done processing 11878
    I1108 19:55:08.034596 28424 compute_partitions.py:192] Done processing 11879
    I1108 19:55:08.378635 28424 compute_partitions.py:192] Done processing 11880
    I1108 19:55:08.729737 28424 compute_partitions.py:192] Done processing 11881
    I1108 19:55:09.090770 28424 compute_partitions.py:192] Done processing 11882
    I1108 19:55:09.423881 28424 compute_partitions.py:192] Done processing 11883
    I1108 19:55:09.766925 28424 compute_partitions.py:192] Done processing 11884
    I1108 19:55:10.127959 28424 compute_partitions.py:192] Done processing 11885
    I1108 19:55:10.470084 28424 compute_partitions.py:192] Done processing 11886
    I1108 19:55:10.816162 28424 compute_partitions.py:192] Done processing 11887
    I1108 19:55:11.170216 28424 compute_partitions.py:192] Done processing 11888
    I1108 19:55:11.513299 28424 compute_partitions.py:192] Done processing 11889
    I1108 19:55:11.858374 28424 compute_partitions.py:192] Done processing 11890
    I1108 19:55:12.220367 28424 compute_partitions.py:192] Done processing 11891
    I1108 19:55:12.565491 28424 compute_partitions.py:192] Done processing 11892
    I1108 19:55:12.909565 28424 compute_partitions.py:192] Done processing 11893
    I1108 19:55:13.260615 28424 compute_partitions.py:192] Done processing 11894
    I1108 19:55:13.623656 28424 compute_partitions.py:192] Done processing 11895
    I1108 19:55:13.984694 28424 compute_partitions.py:192] Done processing 11896
    I1108 19:55:14.333762 28424 compute_partitions.py:192] Done processing 11897
    I1108 19:55:14.679835 28424 compute_partitions.py:192] Done processing 11898
    I1108 19:55:15.033888 28424 compute_partitions.py:192] Done processing 11899
    I1108 19:55:15.378966 28424 compute_partitions.py:192] Done processing 11900
    I1108 19:55:15.725001 28424 compute_partitions.py:192] Done processing 11901
    I1108 19:55:16.070120 28424 compute_partitions.py:192] Done processing 11902
    I1108 19:55:16.416175 28424 compute_partitions.py:192] Done processing 11903
    I1108 19:55:16.763269 28424 compute_partitions.py:192] Done processing 11904
    I1108 19:55:17.121297 28424 compute_partitions.py:192] Done processing 11905
    I1108 19:55:17.469379 28424 compute_partitions.py:192] Done processing 11906
    I1108 19:55:17.825412 28424 compute_partitions.py:192] Done processing 11907
    I1108 19:55:18.189454 28424 compute_partitions.py:192] Done processing 11908
    I1108 19:55:18.530543 28424 compute_partitions.py:192] Done processing 11909
    I1108 19:55:18.874624 28424 compute_partitions.py:192] Done processing 11910
    I1108 19:55:19.229676 28424 compute_partitions.py:192] Done processing 11911
    I1108 19:55:19.575749 28424 compute_partitions.py:192] Done processing 11912
    I1108 19:55:19.932796 28424 compute_partitions.py:192] Done processing 11913
    I1108 19:55:20.289836 28424 compute_partitions.py:192] Done processing 11914
    I1108 19:55:20.635917 28424 compute_partitions.py:192] Done processing 11915
    I1108 19:55:20.981991 28424 compute_partitions.py:192] Done processing 11916
    I1108 19:55:21.320048 28424 compute_partitions.py:192] Done processing 11917
    I1108 19:55:21.666167 28424 compute_partitions.py:192] Done processing 11918
    I1108 19:55:22.016228 28424 compute_partitions.py:192] Done processing 11919
    I1108 19:55:22.361293 28424 compute_partitions.py:192] Done processing 11920
    I1108 19:55:22.717354 28424 compute_partitions.py:192] Done processing 11921
    I1108 19:55:23.076356 28424 compute_partitions.py:192] Done processing 11922
    I1108 19:55:23.420436 28424 compute_partitions.py:192] Done processing 11923
    I1108 19:55:23.771536 28424 compute_partitions.py:192] Done processing 11924
    I1108 19:55:24.122598 28424 compute_partitions.py:192] Done processing 11925
    I1108 19:55:24.455696 28424 compute_partitions.py:192] Done processing 11926
    I1108 19:55:24.800786 28424 compute_partitions.py:192] Done processing 11927
    I1108 19:55:25.153850 28424 compute_partitions.py:192] Done processing 11928
    I1108 19:55:25.494930 28424 compute_partitions.py:192] Done processing 11929
    I1108 19:55:25.840966 28424 compute_partitions.py:192] Done processing 11930
    I1108 19:55:26.198054 28424 compute_partitions.py:192] Done processing 11931
    I1108 19:55:26.540139 28424 compute_partitions.py:192] Done processing 11932
    I1108 19:55:26.902169 28424 compute_partitions.py:192] Done processing 11933
    I1108 19:55:27.253231 28424 compute_partitions.py:192] Done processing 11934
    I1108 19:55:27.603298 28424 compute_partitions.py:192] Done processing 11935
    I1108 19:55:27.948336 28424 compute_partitions.py:192] Done processing 11936
    I1108 19:55:28.291464 28424 compute_partitions.py:192] Done processing 11937
    I1108 19:55:28.656481 28424 compute_partitions.py:192] Done processing 11938
    I1108 19:55:29.002559 28424 compute_partitions.py:192] Done processing 11939
    I1108 19:55:29.353621 28424 compute_partitions.py:192] Done processing 11940
    I1108 19:55:29.713658 28424 compute_partitions.py:192] Done processing 11941
    I1108 19:55:30.076686 28424 compute_partitions.py:192] Done processing 11942
    I1108 19:55:30.409756 28424 compute_partitions.py:192] Done processing 11943
    I1108 19:55:30.755871 28424 compute_partitions.py:192] Done processing 11944
    I1108 19:55:31.112918 28424 compute_partitions.py:192] Done processing 11945
    I1108 19:55:31.443035 28424 compute_partitions.py:192] Done processing 11946
    I1108 19:55:31.795076 28424 compute_partitions.py:192] Done processing 11947
    I1108 19:55:32.148150 28424 compute_partitions.py:192] Done processing 11948
    I1108 19:55:32.496214 28424 compute_partitions.py:192] Done processing 11949
    I1108 19:55:32.845287 28424 compute_partitions.py:192] Done processing 11950
    I1108 19:55:33.200338 28424 compute_partitions.py:192] Done processing 11951
    I1108 19:55:33.539394 28424 compute_partitions.py:192] Done processing 11952
    I1108 19:55:33.885509 28424 compute_partitions.py:192] Done processing 11953
    I1108 19:55:34.242552 28424 compute_partitions.py:192] Done processing 11954
    I1108 19:55:34.596567 28424 compute_partitions.py:192] Done processing 11955
    I1108 19:55:34.945674 28424 compute_partitions.py:192] Done processing 11956
    I1108 19:55:35.292750 28424 compute_partitions.py:192] Done processing 11957
    I1108 19:55:35.650777 28424 compute_partitions.py:192] Done processing 11958
    I1108 19:55:36.000855 28424 compute_partitions.py:192] Done processing 11959
    I1108 19:55:36.346889 28424 compute_partitions.py:192] Done processing 11960
    I1108 19:55:36.692964 28424 compute_partitions.py:192] Done processing 11961
    I1108 19:55:37.043068 28424 compute_partitions.py:192] Done processing 11962
    I1108 19:55:37.379170 28424 compute_partitions.py:192] Done processing 11963
    I1108 19:55:37.724249 28424 compute_partitions.py:192] Done processing 11964
    I1108 19:55:38.074315 28424 compute_partitions.py:192] Done processing 11965
    I1108 19:55:38.419391 28424 compute_partitions.py:192] Done processing 11966
    I1108 19:55:38.781422 28424 compute_partitions.py:192] Done processing 11967
    I1108 19:55:39.145449 28424 compute_partitions.py:192] Done processing 11968
    I1108 19:55:39.486500 28424 compute_partitions.py:192] Done processing 11969
    I1108 19:55:39.838601 28424 compute_partitions.py:192] Done processing 11970
    I1108 19:55:40.203583 28424 compute_partitions.py:192] Done processing 11971
    I1108 19:55:40.555684 28424 compute_partitions.py:192] Done processing 11972
    I1108 19:55:40.916724 28424 compute_partitions.py:192] Done processing 11973
    I1108 19:55:41.267778 28424 compute_partitions.py:192] Done processing 11974
    I1108 19:55:41.622892 28424 compute_partitions.py:192] Done processing 11975
    I1108 19:55:41.982931 28424 compute_partitions.py:192] Done processing 11976
    I1108 19:55:42.323024 28424 compute_partitions.py:192] Done processing 11977
    I1108 19:55:42.671100 28424 compute_partitions.py:192] Done processing 11978
    I1108 19:55:43.018166 28424 compute_partitions.py:192] Done processing 11979
    I1108 19:55:43.355232 28424 compute_partitions.py:192] Done processing 11980
    I1108 19:55:43.700340 28424 compute_partitions.py:192] Done processing 11981
    I1108 19:55:44.048410 28424 compute_partitions.py:192] Done processing 11982
    I1108 19:55:44.391494 28424 compute_partitions.py:192] Done processing 11983
    I1108 19:55:44.733581 28424 compute_partitions.py:192] Done processing 11984
    I1108 19:55:45.089587 28424 compute_partitions.py:192] Done processing 11985
    I1108 19:55:45.431722 28424 compute_partitions.py:192] Done processing 11986
    I1108 19:55:45.773800 28424 compute_partitions.py:192] Done processing 11987
    I1108 19:55:46.132821 28424 compute_partitions.py:192] Done processing 11988
    I1108 19:55:46.478914 28424 compute_partitions.py:192] Done processing 11989
    I1108 19:55:46.825987 28424 compute_partitions.py:192] Done processing 11990
    I1108 19:55:47.186025 28424 compute_partitions.py:192] Done processing 11991
    I1108 19:55:47.524121 28424 compute_partitions.py:192] Done processing 11992
    I1108 19:55:47.890143 28424 compute_partitions.py:192] Done processing 11993
    I1108 19:55:48.260158 28424 compute_partitions.py:192] Done processing 11994
    I1108 19:55:48.605253 28424 compute_partitions.py:192] Done processing 11995
    I1108 19:55:48.955317 28424 compute_partitions.py:192] Done processing 11996
    I1108 19:55:49.302428 28424 compute_partitions.py:192] Done processing 11997
    I1108 19:55:49.645511 28424 compute_partitions.py:192] Done processing 11998
    I1108 19:55:49.992545 28424 compute_partitions.py:192] Done processing 11999
    I1108 19:55:50.332678 28424 compute_partitions.py:192] Done processing 12000
    I1108 19:55:50.676757 28424 compute_partitions.py:192] Done processing 12001
    I1108 19:55:51.025822 28424 compute_partitions.py:192] Done processing 12002
    I1108 19:55:51.364904 28424 compute_partitions.py:192] Done processing 12003
    I1108 19:55:51.720926 28424 compute_partitions.py:192] Done processing 12004
    I1108 19:55:52.075977 28424 compute_partitions.py:192] Done processing 12005
    I1108 19:55:52.424086 28424 compute_partitions.py:192] Done processing 12006
    I1108 19:55:52.769168 28424 compute_partitions.py:192] Done processing 12007
    I1108 19:55:53.127168 28424 compute_partitions.py:192] Done processing 12008
    I1108 19:55:53.472244 28424 compute_partitions.py:192] Done processing 12009
    I1108 19:55:53.816357 28424 compute_partitions.py:192] Done processing 12010
    I1108 19:55:54.176392 28424 compute_partitions.py:192] Done processing 12011
    I1108 19:55:54.518477 28424 compute_partitions.py:192] Done processing 12012
    I1108 19:55:54.871545 28424 compute_partitions.py:192] Done processing 12013
    I1108 19:55:55.225586 28424 compute_partitions.py:192] Done processing 12014
    I1108 19:55:55.568669 28424 compute_partitions.py:192] Done processing 12015
    I1108 19:55:55.920733 28424 compute_partitions.py:192] Done processing 12016
    I1108 19:55:56.271802 28424 compute_partitions.py:192] Done processing 12017
    I1108 19:55:56.628809 28424 compute_partitions.py:192] Done processing 12018
    I1108 19:55:56.980907 28424 compute_partitions.py:192] Done processing 12019
    I1108 19:55:57.344897 28424 compute_partitions.py:192] Done processing 12020
    I1108 19:55:57.711955 28424 compute_partitions.py:192] Done processing 12021
    I1108 19:55:58.056038 28424 compute_partitions.py:192] Done processing 12022
    I1108 19:55:58.408095 28424 compute_partitions.py:192] Done processing 12023
    I1108 19:55:58.757153 28424 compute_partitions.py:192] Done processing 12024
    I1108 19:55:59.116182 28424 compute_partitions.py:192] Done processing 12025
    I1108 19:55:59.452302 28424 compute_partitions.py:192] Done processing 12026
    I1108 19:55:59.794392 28424 compute_partitions.py:192] Done processing 12027
    I1108 19:56:00.167353 28424 compute_partitions.py:192] Done processing 12028
    I1108 19:56:00.507443 28424 compute_partitions.py:192] Done processing 12029
    I1108 19:56:00.875499 28424 compute_partitions.py:192] Done processing 12030
    I1108 19:56:01.230511 28424 compute_partitions.py:192] Done processing 12031
    I1108 19:56:01.576625 28424 compute_partitions.py:192] Done processing 12032
    I1108 19:56:01.919709 28424 compute_partitions.py:192] Done processing 12033
    I1108 19:56:02.270770 28424 compute_partitions.py:192] Done processing 12034
    I1108 19:56:02.619825 28424 compute_partitions.py:192] Done processing 12035
    I1108 19:56:02.969863 28424 compute_partitions.py:192] Done processing 12036
    I1108 19:56:03.316936 28424 compute_partitions.py:192] Done processing 12037
    I1108 19:56:03.660058 28424 compute_partitions.py:192] Done processing 12038
    I1108 19:56:04.008122 28424 compute_partitions.py:192] Done processing 12039
    I1108 19:56:04.355201 28424 compute_partitions.py:192] Done processing 12040
    I1108 19:56:04.699268 28424 compute_partitions.py:192] Done processing 12042
    I1108 19:56:04.706264 28424 compute_partitions.py:194] Nonzero values: 3692656

Inputs and outputs of Script 2 in cmd.exe:

    python build_coordinates.py --partition_volumes validation1:brainlit/images/test_1/af.h5:af --coordinate_output brainlit/images/test_1/tf_record_file --margin 24,24,24
    I1108 20:04:07.356520 10340 build_coordinates.py:78] Partition counts:
    I1108 20:04:07.356520 10340 build_coordinates.py:80]  0: 442592
    I1108 20:04:07.357479 10340 build_coordinates.py:80]  1: 3692656
    I1108 20:04:07.357479 10340 build_coordinates.py:82] Resampling and shuffling coordinates.
    I1108 20:04:19.047237 10340 build_coordinates.py:90] Saving coordinates.

Train the model (run the following script in cmd.exe)

Script 3:

    python train.py \
        --train_coords brainlit/images/test_1/tf_record_file \
        --data_volumes validation1:brainlit/images/test_1/grayscale_maps.h5:raw \
        --label_volumes validation1:brainlit/images/test_1/groundtruth.h5:stack \
        --model_name convstack_3d.ConvStack3DFFNModel \
        --model_args "{\"depth\": 12, \"fov_size\": [33, 33, 33], \"deltas\": [8, 8, 8]}" \
        --image_mean 128 \
        --image_stddev 33

Inputs and outputs of Script 3 in cmd.exe:

    python train.py --train_coords brainlit/images/test_1/tf_record_file --data_volumes validation1:brainlit/images/test_1/grayscale_maps.h5:raw --label_volumes validation1:brainlit/images/test_1/groundtruth.h5:stack --model_name convstack_3d.ConvStack3DFFNModel --model_args "{\"depth\": 12, \"fov_size\": [33, 33, 33], \"deltas\": [8, 8, 8]}" --image_mean 128 --image_stddev 33
    
    python train.py --train_coords brainlit/images/test_1/tf_record_file --data_volumes validation1:brainlit/images/test_1/grayscale_maps.h5:raw --label_volumes validation1:brainlit/images/test_1/groundtruth.h5:stack --model_name convstack_3d.ConvStack3DFFNModel --model_args "{\"depth\": 6, \"fov_size\": [33, 33, 11], \"deltas\": [8, 8, 8]}" --image_mean 128 --image_stddev 33
    
    python train.py --train_coords brainlit/images/test_1/tf_record_file --data_volumes validation1:brainlit/images/test_1/grayscale_maps.h5:raw --label_volumes validation1:brainlit/images/test_1/groundtruth.h5:stack --model_name convstack_3d.ConvStack3DFFNModel --model_args "{\"depth\": 6, \"fov_size\": [11, 11, 11], \"deltas\": [4, 4, 4]}" --image_mean 64 --image_stddev 33

    Traceback (most recent call last):
      File "train.py", line 734, in <module>
        app.run(main)
      File "D:\Anaconda3\envs\brainlit\lib\site-packages\absl\app.py", line 300, in run
        _run_main(main, args)
      File "D:\Anaconda3\envs\brainlit\lib\site-packages\absl\app.py", line 251, in _run_main
        sys.exit(main(argv))
      File "train.py", line 725, in main
        **json.loads(FLAGS.model_args))
      File "train.py", line 624, in train_ffn
        load_data_ops = define_data_input(model, queue_batch=1)
      File "train.py", line 412, in define_data_input
        0]))
      File "D:\Anaconda3\envs\brainlit\lib\site-packages\tensorflow_core\python\util\dispatch.py", line 180, in wrapper
        return target(*args, **kwargs)
      File "D:\Anaconda3\envs\brainlit\lib\site-packages\tensorflow_core\python\ops\math_ops.py", line 1306, in equal
        return gen_math_ops.equal(x, y, name=name)
      File "D:\Anaconda3\envs\brainlit\lib\site-packages\tensorflow_core\python\ops\gen_math_ops.py", line 3626, in equal
        name=name)
      File "D:\Anaconda3\envs\brainlit\lib\site-packages\tensorflow_core\python\framework\op_def_library.py", line 632, in _apply_op_helper
        param_name=input_name)
      File "D:\Anaconda3\envs\brainlit\lib\site-packages\tensorflow_core\python\framework\op_def_library.py", line 61, in _SatisfiesTypeConstraint
        ", ".join(dtypes.as_dtype(x).name for x in allowed_list)))
    TypeError: Value passed to parameter 'x' has DataType uint16 not in list of allowed values: bfloat16, float16, float32, float64, uint8, int8, int16, int32, int64, complex64, quint8, qint8, qint32, string, bool, complex128
    
Fixed the issue above (see details at the bottom of cell 4) and get the outputs (ignored the warnings):

    I1110 10:24:27.975465 22704 train.py:699] Saving summaries.
    I1110 10:26:29.906550 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.38974
    I1110 10:26:40.243637 22704 basic_session_run_hooks.py:692] global_step/sec: 0.38974
    INFO:tensorflow:Saving checkpoints for 118 into tmp\model.ckpt.
    I1110 10:27:25.611796 22704 basic_session_run_hooks.py:606] Saving checkpoints for 118 into tmp\model.ckpt.
    I1110 10:28:31.887199 22704 train.py:699] Saving summaries.
    I1110 10:30:34.186578 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.40621
    I1110 10:30:46.421684 22704 basic_session_run_hooks.py:692] global_step/sec: 0.40621
    INFO:tensorflow:Saving checkpoints for 240 into tmp\model.ckpt.
    I1110 10:32:25.621470 22704 basic_session_run_hooks.py:606] Saving checkpoints for 240 into tmp\model.ckpt.
    I1110 10:32:35.394906 22704 train.py:699] Saving summaries.
    I1110 10:34:35.456563 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.406465
    I1110 10:34:52.445039 22704 basic_session_run_hooks.py:692] global_step/sec: 0.406465
    I1110 10:36:36.860423 22704 train.py:699] Saving summaries.
    INFO:tensorflow:Saving checkpoints for 362 into tmp\model.ckpt.
    I1110 10:37:26.561062 22704 basic_session_run_hooks.py:606] Saving checkpoints for 362 into tmp\model.ckpt.
    I1110 10:38:38.076722 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.403666
    I1110 10:39:00.175388 22704 basic_session_run_hooks.py:692] global_step/sec: 0.403666
    I1110 10:40:38.878493 22704 train.py:699] Saving summaries.
    INFO:tensorflow:Saving checkpoints for 484 into tmp\model.ckpt.
    I1110 10:42:28.314060 22704 basic_session_run_hooks.py:606] Saving checkpoints for 484 into tmp\model.ckpt.
    I1110 10:42:40.711359 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.403796
    I1110 10:43:07.824315 22704 basic_session_run_hooks.py:692] global_step/sec: 0.403796
    I1110 10:44:41.912303 22704 train.py:699] Saving summaries.
    I1110 10:46:42.702375 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.392578
    I1110 10:47:22.551799 22704 basic_session_run_hooks.py:692] global_step/sec: 0.392578
    INFO:tensorflow:Saving checkpoints for 603 into tmp\model.ckpt.
    I1110 10:47:29.990858 22704 basic_session_run_hooks.py:606] Saving checkpoints for 603 into tmp\model.ckpt.
    I1110 10:48:43.430331 22704 train.py:699] Saving summaries.
    I1110 10:50:43.984158 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.376969
    I1110 10:51:47.824529 22704 basic_session_run_hooks.py:692] global_step/sec: 0.376969
    INFO:tensorflow:Saving checkpoints for 717 into tmp\model.ckpt.
    I1110 10:52:32.335402 22704 basic_session_run_hooks.py:606] Saving checkpoints for 717 into tmp\model.ckpt.
    I1110 10:52:46.331940 22704 train.py:699] Saving summaries.
    I1110 10:54:48.173473 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.358717
    I1110 10:56:26.596082 22704 basic_session_run_hooks.py:692] global_step/sec: 0.358717
    I1110 10:56:48.467015 22704 train.py:699] Saving summaries.
    INFO:tensorflow:Saving checkpoints for 826 into tmp\model.ckpt.
    I1110 10:57:34.046776 22704 basic_session_run_hooks.py:606] Saving checkpoints for 826 into tmp\model.ckpt.
    I1110 10:58:50.832880 22704 train.py:699] Saving summaries.
    I1110 11:00:50.159794 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.369503
    I1110 11:00:57.230019 22704 basic_session_run_hooks.py:692] global_step/sec: 0.369503
    INFO:tensorflow:Saving checkpoints for 938 into tmp\model.ckpt.
    I1110 11:02:36.223836 22704 basic_session_run_hooks.py:606] Saving checkpoints for 938 into tmp\model.ckpt.
    I1110 11:02:51.341502 22704 train.py:699] Saving summaries.
    I1110 11:04:52.086885 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.355546
    I1110 11:05:38.487492 22704 basic_session_run_hooks.py:692] global_step/sec: 0.355546
    I1110 11:06:53.043019 22704 train.py:699] Saving summaries.
    INFO:tensorflow:Saving checkpoints for 1044 into tmp\model.ckpt.
    I1110 11:07:37.200545 22704 basic_session_run_hooks.py:606] Saving checkpoints for 1044 into tmp\model.ckpt.
    I1110 11:08:53.754865 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.360256
    I1110 11:10:16.067983 22704 basic_session_run_hooks.py:692] global_step/sec: 0.360256
    I1110 11:10:57.481664 22704 train.py:699] Saving summaries.
    INFO:tensorflow:Saving checkpoints for 1149 into tmp\model.ckpt.
    I1110 11:12:38.015846 22704 basic_session_run_hooks.py:606] Saving checkpoints for 1149 into tmp\model.ckpt.
    I1110 11:12:56.934346 22704 train.py:699] Saving summaries.
    I1110 11:14:57.292800 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.348959
    I1110 11:15:02.634266 22704 basic_session_run_hooks.py:692] global_step/sec: 0.348959
    I1110 11:16:57.894780 22704 train.py:699] Saving summaries.
    INFO:tensorflow:Saving checkpoints for 1268 into tmp\model.ckpt.
    I1110 11:17:39.814049 22704 basic_session_run_hooks.py:606] Saving checkpoints for 1268 into tmp\model.ckpt.
    INFO:tensorflow:global_step/sec: 0.438819
    I1110 11:18:50.518443 22704 basic_session_run_hooks.py:692] global_step/sec: 0.438819
    I1110 11:18:59.334961 22704 train.py:699] Saving summaries.
    I1110 11:21:01.455287 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.451281
    I1110 11:22:32.111011 22704 basic_session_run_hooks.py:692] global_step/sec: 0.451281
    INFO:tensorflow:Saving checkpoints for 1404 into tmp\model.ckpt.
    I1110 11:22:40.986377 22704 basic_session_run_hooks.py:606] Saving checkpoints for 1404 into tmp\model.ckpt.
    I1110 11:23:03.165082 22704 train.py:699] Saving summaries.
    I1110 11:25:04.692508 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.452818
    I1110 11:26:12.949282 22704 basic_session_run_hooks.py:692] global_step/sec: 0.452818
    I1110 11:27:05.790197 22704 train.py:699] Saving summaries.
    INFO:tensorflow:Saving checkpoints for 1540 into tmp\model.ckpt.
    I1110 11:27:40.988266 22704 basic_session_run_hooks.py:606] Saving checkpoints for 1540 into tmp\model.ckpt.
    I1110 11:29:06.846021 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.453902
    I1110 11:29:53.261111 22704 basic_session_run_hooks.py:692] global_step/sec: 0.453902
    I1110 11:31:08.375336 22704 train.py:699] Saving summaries.
    INFO:tensorflow:Saving checkpoints for 1677 into tmp\model.ckpt.
    I1110 11:32:42.987429 22704 basic_session_run_hooks.py:606] Saving checkpoints for 1677 into tmp\model.ckpt.
    I1110 11:33:09.494609 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.453709
    I1110 11:33:33.666543 22704 basic_session_run_hooks.py:692] global_step/sec: 0.453709
    I1110 11:35:10.600269 22704 train.py:699] Saving summaries.
    I1110 11:37:11.903004 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.453641
    I1110 11:37:14.105139 22704 basic_session_run_hooks.py:692] global_step/sec: 0.453641
    INFO:tensorflow:Saving checkpoints for 1814 into tmp\model.ckpt.
    I1110 11:37:44.909794 22704 basic_session_run_hooks.py:606] Saving checkpoints for 1814 into tmp\model.ckpt.
    I1110 11:39:12.898944 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.453849
    I1110 11:40:54.442705 22704 basic_session_run_hooks.py:692] global_step/sec: 0.453849
    I1110 11:41:14.324253 22704 train.py:699] Saving summaries.
    INFO:tensorflow:Saving checkpoints for 1951 into tmp\model.ckpt.
    I1110 11:42:46.930515 22704 basic_session_run_hooks.py:606] Saving checkpoints for 1951 into tmp\model.ckpt.
    I1110 11:43:15.650803 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.453898
    I1110 11:44:34.756545 22704 basic_session_run_hooks.py:692] global_step/sec: 0.453898
    I1110 11:45:16.610216 22704 train.py:699] Saving summaries.
    I1110 11:47:17.684217 22704 train.py:699] Saving summaries.
    INFO:tensorflow:Saving checkpoints for 2088 into tmp\model.ckpt.
    I1110 11:47:48.475956 22704 basic_session_run_hooks.py:606] Saving checkpoints for 2088 into tmp\model.ckpt.
    INFO:tensorflow:global_step/sec: 0.454106
    I1110 11:48:14.969415 22704 basic_session_run_hooks.py:692] global_step/sec: 0.454106
    I1110 11:49:18.697754 22704 train.py:699] Saving summaries.
    I1110 11:51:19.970429 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.453993
    I1110 11:51:55.236909 22704 basic_session_run_hooks.py:692] global_step/sec: 0.453993
    INFO:tensorflow:Saving checkpoints for 2225 into tmp\model.ckpt.
    I1110 11:52:50.414915 22704 basic_session_run_hooks.py:606] Saving checkpoints for 2225 into tmp\model.ckpt.
    I1110 11:53:21.238662 22704 train.py:699] Saving summaries.
    I1110 11:55:22.167065 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.454211
    I1110 11:55:35.398705 22704 basic_session_run_hooks.py:692] global_step/sec: 0.454211
    I1110 11:57:23.620016 22704 train.py:699] Saving summaries.
    INFO:tensorflow:Saving checkpoints for 2362 into tmp\model.ckpt.
    I1110 11:57:52.211713 22704 basic_session_run_hooks.py:606] Saving checkpoints for 2362 into tmp\model.ckpt.
    INFO:tensorflow:global_step/sec: 0.45333
    I1110 11:59:15.989683 22704 basic_session_run_hooks.py:692] global_step/sec: 0.45333
    I1110 11:59:24.786204 22704 train.py:699] Saving summaries.
    I1110 12:01:25.900359 22704 train.py:699] Saving summaries.
    INFO:tensorflow:Saving checkpoints for 2499 into tmp\model.ckpt.
    I1110 12:02:53.877512 22704 basic_session_run_hooks.py:606] Saving checkpoints for 2499 into tmp\model.ckpt.
    INFO:tensorflow:global_step/sec: 0.454211
    I1110 12:02:56.150480 22704 basic_session_run_hooks.py:692] global_step/sec: 0.454211
    I1110 12:03:26.972546 22704 train.py:699] Saving summaries.
    I1110 12:05:27.878248 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.454357
    I1110 12:06:36.241949 22704 basic_session_run_hooks.py:692] global_step/sec: 0.454357
    I1110 12:07:29.167565 22704 train.py:699] Saving summaries.
    INFO:tensorflow:Saving checkpoints for 2636 into tmp\model.ckpt.
    I1110 12:07:55.540157 22704 basic_session_run_hooks.py:606] Saving checkpoints for 2636 into tmp\model.ckpt.
    I1110 12:09:29.356360 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.449655
    I1110 12:10:18.634820 22704 basic_session_run_hooks.py:692] global_step/sec: 0.449655
    I1110 12:11:29.848483 22704 train.py:699] Saving summaries.
    INFO:tensorflow:Saving checkpoints for 2771 into tmp\model.ckpt.
    I1110 12:12:56.445137 22704 basic_session_run_hooks.py:606] Saving checkpoints for 2771 into tmp\model.ckpt.
    I1110 12:13:31.689852 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.395438
    I1110 12:14:31.520145 22704 basic_session_run_hooks.py:692] global_step/sec: 0.395438
    I1110 12:15:31.573347 22704 train.py:699] Saving summaries.
    I1110 12:17:32.307740 22704 train.py:699] Saving summaries.
    INFO:tensorflow:Saving checkpoints for 2884 into tmp\model.ckpt.
    I1110 12:17:57.175877 22704 basic_session_run_hooks.py:606] Saving checkpoints for 2884 into tmp\model.ckpt.
    INFO:tensorflow:global_step/sec: 0.407476
    I1110 12:18:36.932334 22704 basic_session_run_hooks.py:692] global_step/sec: 0.407476
    I1110 12:19:32.754993 22704 train.py:699] Saving summaries.
    I1110 12:21:34.204809 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.411497
    I1110 12:22:39.947597 22704 basic_session_run_hooks.py:692] global_step/sec: 0.411497
    INFO:tensorflow:Saving checkpoints for 3008 into tmp\model.ckpt.
    I1110 12:22:59.033922 22704 basic_session_run_hooks.py:606] Saving checkpoints for 3008 into tmp\model.ckpt.
    I1110 12:23:34.548300 22704 train.py:699] Saving summaries.
    I1110 12:25:36.357519 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.4067
    I1110 12:26:45.829295 22704 basic_session_run_hooks.py:692] global_step/sec: 0.4067
    I1110 12:27:36.964313 22704 train.py:699] Saving summaries.
    INFO:tensorflow:Saving checkpoints for 3132 into tmp\model.ckpt.
    I1110 12:28:00.613304 22704 basic_session_run_hooks.py:606] Saving checkpoints for 3132 into tmp\model.ckpt.
    I1110 12:29:38.068595 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.383883
    I1110 12:31:06.325158 22704 basic_session_run_hooks.py:692] global_step/sec: 0.383883
    I1110 12:31:39.828966 22704 train.py:699] Saving summaries.
    INFO:tensorflow:Saving checkpoints for 3249 into tmp\model.ckpt.
    I1110 12:33:02.310817 22704 basic_session_run_hooks.py:606] Saving checkpoints for 3249 into tmp\model.ckpt.
    I1110 12:33:41.895518 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.404183
    I1110 12:35:13.737974 22704 basic_session_run_hooks.py:692] global_step/sec: 0.404183
    I1110 12:35:43.763267 22704 train.py:699] Saving summaries.
    I1110 12:37:43.659024 22704 train.py:699] Saving summaries.
    INFO:tensorflow:Saving checkpoints for 3359 into tmp\model.ckpt.
    I1110 12:38:03.142333 22704 basic_session_run_hooks.py:606] Saving checkpoints for 3359 into tmp\model.ckpt.
    I1110 12:39:45.429770 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.359021
    I1110 12:39:52.273165 22704 basic_session_run_hooks.py:692] global_step/sec: 0.359021
    I1110 12:41:46.210486 22704 train.py:699] Saving summaries.
    INFO:tensorflow:Saving checkpoints for 3484 into tmp\model.ckpt.
    I1110 12:43:03.515936 22704 basic_session_run_hooks.py:606] Saving checkpoints for 3484 into tmp\model.ckpt.
    INFO:tensorflow:global_step/sec: 0.43854
    I1110 12:43:40.302348 22704 basic_session_run_hooks.py:692] global_step/sec: 0.43854
    I1110 12:43:47.170151 22704 train.py:699] Saving summaries.
    I1110 12:45:47.204545 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.442556
    I1110 12:47:26.262634 22704 basic_session_run_hooks.py:692] global_step/sec: 0.442556
    I1110 12:47:49.054934 22704 train.py:699] Saving summaries.
    INFO:tensorflow:Saving checkpoints for 3617 into tmp\model.ckpt.
    I1110 12:48:04.974151 22704 basic_session_run_hooks.py:606] Saving checkpoints for 3617 into tmp\model.ckpt.
    I1110 12:49:50.540547 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.43555
    I1110 12:51:15.857592 22704 basic_session_run_hooks.py:692] global_step/sec: 0.43555
    I1110 12:51:52.554890 22704 train.py:699] Saving summaries.
    INFO:tensorflow:Saving checkpoints for 3748 into tmp\model.ckpt.
    I1110 12:53:06.977941 22704 basic_session_run_hooks.py:606] Saving checkpoints for 3748 into tmp\model.ckpt.
    I1110 12:53:53.856363 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.418142
    I1110 12:55:15.011026 22704 basic_session_run_hooks.py:692] global_step/sec: 0.418142
    I1110 12:55:53.827275 22704 train.py:699] Saving summaries.
    I1110 12:57:54.740554 22704 train.py:699] Saving summaries.
    INFO:tensorflow:Saving checkpoints for 3873 into tmp\model.ckpt.
    I1110 12:58:06.985623 22704 basic_session_run_hooks.py:606] Saving checkpoints for 3873 into tmp\model.ckpt.
    INFO:tensorflow:global_step/sec: 0.417992
    I1110 12:59:14.251039 22704 basic_session_run_hooks.py:692] global_step/sec: 0.417992
    I1110 12:59:55.549879 22704 train.py:699] Saving summaries.
    I1110 13:01:58.665194 22704 train.py:699] Saving summaries.
    INFO:tensorflow:Saving checkpoints for 3995 into tmp\model.ckpt.
    I1110 13:03:08.662267 22704 basic_session_run_hooks.py:606] Saving checkpoints for 3995 into tmp\model.ckpt.
    INFO:tensorflow:global_step/sec: 0.405244
    I1110 13:03:21.014871 22704 basic_session_run_hooks.py:692] global_step/sec: 0.405244
    I1110 13:03:57.727495 22704 train.py:699] Saving summaries.
    I1110 13:05:59.304128 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.405255
    I1110 13:07:27.774007 22704 basic_session_run_hooks.py:692] global_step/sec: 0.405255
    I1110 13:07:59.645617 22704 train.py:699] Saving summaries.
    INFO:tensorflow:Saving checkpoints for 4117 into tmp\model.ckpt.
    I1110 13:08:09.858441 22704 basic_session_run_hooks.py:606] Saving checkpoints for 4117 into tmp\model.ckpt.
    I1110 13:10:00.287419 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.407562
    I1110 13:11:33.135489 22704 basic_session_run_hooks.py:692] global_step/sec: 0.407562
    I1110 13:12:02.333792 22704 train.py:699] Saving summaries.
    INFO:tensorflow:Saving checkpoints for 4240 into tmp\model.ckpt.
    I1110 13:13:11.453792 22704 basic_session_run_hooks.py:606] Saving checkpoints for 4240 into tmp\model.ckpt.
    I1110 13:14:03.184135 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.401895
    I1110 13:15:41.955872 22704 basic_session_run_hooks.py:692] global_step/sec: 0.401895
    I1110 13:16:03.492680 22704 train.py:699] Saving summaries.
    I1110 13:18:04.594255 22704 train.py:699] Saving summaries.
    INFO:tensorflow:Saving checkpoints for 4362 into tmp\model.ckpt.
    I1110 13:18:12.083165 22704 basic_session_run_hooks.py:606] Saving checkpoints for 4362 into tmp\model.ckpt.
    INFO:tensorflow:global_step/sec: 0.411495
    I1110 13:19:44.972318 22704 basic_session_run_hooks.py:692] global_step/sec: 0.411495
    I1110 13:20:06.681536 22704 train.py:699] Saving summaries.
    I1110 13:22:08.472656 22704 train.py:699] Saving summaries.
    INFO:tensorflow:Saving checkpoints for 4486 into tmp\model.ckpt.
    I1110 13:23:13.821272 22704 basic_session_run_hooks.py:606] Saving checkpoints for 4486 into tmp\model.ckpt.
    INFO:tensorflow:global_step/sec: 0.412232
    I1110 13:23:47.553886 22704 basic_session_run_hooks.py:692] global_step/sec: 0.412232
    I1110 13:24:09.304004 22704 train.py:699] Saving summaries.
    I1110 13:26:09.970801 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.413604
    I1110 13:27:49.332060 22704 basic_session_run_hooks.py:692] global_step/sec: 0.413604
    I1110 13:28:11.058957 22704 train.py:699] Saving summaries.
    INFO:tensorflow:Saving checkpoints for 4611 into tmp\model.ckpt.
    I1110 13:28:15.849698 22704 basic_session_run_hooks.py:606] Saving checkpoints for 4611 into tmp\model.ckpt.
    I1110 13:30:12.868722 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.411982
    I1110 13:31:52.060275 22704 basic_session_run_hooks.py:692] global_step/sec: 0.411982
    I1110 13:32:13.877886 22704 train.py:699] Saving summaries.
    INFO:tensorflow:Saving checkpoints for 4735 into tmp\model.ckpt.
    I1110 13:33:17.065137 22704 basic_session_run_hooks.py:606] Saving checkpoints for 4735 into tmp\model.ckpt.
    I1110 13:34:16.098165 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.400306
    I1110 13:36:01.869346 22704 basic_session_run_hooks.py:692] global_step/sec: 0.400306
    I1110 13:36:16.823409 22704 train.py:699] Saving summaries.
    INFO:tensorflow:Saving checkpoints for 4856 into tmp\model.ckpt.
    I1110 13:38:17.778757 22704 basic_session_run_hooks.py:606] Saving checkpoints for 4856 into tmp\model.ckpt.
    I1110 13:38:17.859761 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.410471
    I1110 13:40:05.492212 22704 basic_session_run_hooks.py:692] global_step/sec: 0.410471
    I1110 13:40:18.020957 22704 train.py:699] Saving summaries.
    I1110 13:42:19.893343 22704 train.py:699] Saving summaries.
    INFO:tensorflow:Saving checkpoints for 4979 into tmp\model.ckpt.
    I1110 13:43:18.379708 22704 basic_session_run_hooks.py:606] Saving checkpoints for 4979 into tmp\model.ckpt.
    INFO:tensorflow:global_step/sec: 0.408952
    I1110 13:44:10.019696 22704 basic_session_run_hooks.py:692] global_step/sec: 0.408952
    I1110 13:44:22.116312 22704 train.py:699] Saving summaries.
    I1110 13:46:24.397619 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.410177
    I1110 13:48:13.816691 22704 basic_session_run_hooks.py:692] global_step/sec: 0.410177
    INFO:tensorflow:Saving checkpoints for 5102 into tmp\model.ckpt.
    I1110 13:48:18.682022 22704 basic_session_run_hooks.py:606] Saving checkpoints for 5102 into tmp\model.ckpt.
    I1110 13:48:26.131708 22704 train.py:699] Saving summaries.
    I1110 13:50:27.661010 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.413098
    I1110 13:52:15.890173 22704 basic_session_run_hooks.py:692] global_step/sec: 0.413098
    I1110 13:52:27.842629 22704 train.py:699] Saving summaries.
    INFO:tensorflow:Saving checkpoints for 5227 into tmp\model.ckpt.
    I1110 13:53:20.659904 22704 basic_session_run_hooks.py:606] Saving checkpoints for 5227 into tmp\model.ckpt.
    I1110 13:54:28.041135 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.416687
    I1110 13:56:15.878223 22704 basic_session_run_hooks.py:692] global_step/sec: 0.416687
    I1110 13:56:30.298247 22704 train.py:699] Saving summaries.
    INFO:tensorflow:Saving checkpoints for 5352 into tmp\model.ckpt.
    I1110 13:58:21.151396 22704 basic_session_run_hooks.py:606] Saving checkpoints for 5352 into tmp\model.ckpt.
    I1110 13:58:30.877695 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.415918
    I1110 14:00:16.310106 22704 basic_session_run_hooks.py:692] global_step/sec: 0.415918
    I1110 14:00:33.186011 22704 train.py:699] Saving summaries.
    I1110 14:02:34.146492 22704 train.py:699] Saving summaries.
    INFO:tensorflow:Saving checkpoints for 5478 into tmp\model.ckpt.
    I1110 14:03:23.197455 22704 basic_session_run_hooks.py:606] Saving checkpoints for 5478 into tmp\model.ckpt.
    INFO:tensorflow:global_step/sec: 0.425049
    I1110 14:04:11.577123 22704 basic_session_run_hooks.py:692] global_step/sec: 0.425049
    I1110 14:04:35.732593 22704 train.py:699] Saving summaries.
    I1110 14:06:36.628697 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.435212
    I1110 14:08:01.350283 22704 basic_session_run_hooks.py:692] global_step/sec: 0.435212
    INFO:tensorflow:Saving checkpoints for 5610 into tmp\model.ckpt.
    I1110 14:08:24.708486 22704 basic_session_run_hooks.py:606] Saving checkpoints for 5610 into tmp\model.ckpt.
    I1110 14:08:36.721617 22704 train.py:699] Saving summaries.
    I1110 14:10:36.998793 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.424876
    I1110 14:11:56.712806 22704 basic_session_run_hooks.py:692] global_step/sec: 0.424876
    I1110 14:12:37.229211 22704 train.py:699] Saving summaries.
    INFO:tensorflow:Saving checkpoints for 5737 into tmp\model.ckpt.
    I1110 14:13:24.828274 22704 basic_session_run_hooks.py:606] Saving checkpoints for 5737 into tmp\model.ckpt.
    I1110 14:14:38.677056 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.42074
    I1110 14:15:54.389513 22704 basic_session_run_hooks.py:692] global_step/sec: 0.42074
    I1110 14:16:39.881650 22704 train.py:699] Saving summaries.
    INFO:tensorflow:Saving checkpoints for 5864 into tmp\model.ckpt.
    I1110 14:18:27.061617 22704 basic_session_run_hooks.py:606] Saving checkpoints for 5864 into tmp\model.ckpt.
    I1110 14:18:41.300388 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.419114
    I1110 14:19:52.988179 22704 basic_session_run_hooks.py:692] global_step/sec: 0.419114
    I1110 14:20:43.039820 22704 train.py:699] Saving summaries.
    I1110 14:22:44.502285 22704 train.py:699] Saving summaries.
    INFO:tensorflow:Saving checkpoints for 5990 into tmp\model.ckpt.
    I1110 14:23:27.748219 22704 basic_session_run_hooks.py:606] Saving checkpoints for 5990 into tmp\model.ckpt.
    INFO:tensorflow:global_step/sec: 0.418427
    I1110 14:23:51.978427 22704 basic_session_run_hooks.py:692] global_step/sec: 0.418427
    I1110 14:24:46.528699 22704 train.py:699] Saving summaries.
    I1110 14:26:47.885176 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.417415
    I1110 14:27:51.548270 22704 basic_session_run_hooks.py:692] global_step/sec: 0.417415
    INFO:tensorflow:Saving checkpoints for 6116 into tmp\model.ckpt.
    I1110 14:28:28.728963 22704 basic_session_run_hooks.py:606] Saving checkpoints for 6116 into tmp\model.ckpt.
    I1110 14:28:49.800923 22704 train.py:699] Saving summaries.
    I1110 14:30:50.585510 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.430251
    I1110 14:31:43.970900 22704 basic_session_run_hooks.py:692] global_step/sec: 0.430251
    I1110 14:32:52.232494 22704 train.py:699] Saving summaries.
    INFO:tensorflow:Saving checkpoints for 6244 into tmp\model.ckpt.
    I1110 14:33:29.452057 22704 basic_session_run_hooks.py:606] Saving checkpoints for 6244 into tmp\model.ckpt.
    I1110 14:34:53.028820 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.424884
    I1110 14:35:39.329153 22704 basic_session_run_hooks.py:692] global_step/sec: 0.424884
    I1110 14:36:53.481253 22704 train.py:699] Saving summaries.
    INFO:tensorflow:Saving checkpoints for 6374 into tmp\model.ckpt.
    I1110 14:38:31.195425 22704 basic_session_run_hooks.py:606] Saving checkpoints for 6374 into tmp\model.ckpt.
    I1110 14:38:54.602008 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.430147
    I1110 14:39:31.807574 22704 basic_session_run_hooks.py:692] global_step/sec: 0.430147
    I1110 14:40:55.672626 22704 train.py:699] Saving summaries.
    I1110 14:42:56.798117 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.430189
    I1110 14:43:24.263376 22704 basic_session_run_hooks.py:692] global_step/sec: 0.430189
    INFO:tensorflow:Saving checkpoints for 6504 into tmp\model.ckpt.
    I1110 14:43:33.156662 22704 basic_session_run_hooks.py:606] Saving checkpoints for 6504 into tmp\model.ckpt.
    I1110 14:44:58.276848 22704 train.py:699] Saving summaries.
    I1110 14:46:58.677479 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.447695
    I1110 14:47:07.629553 22704 basic_session_run_hooks.py:692] global_step/sec: 0.447695
    INFO:tensorflow:Saving checkpoints for 6639 into tmp\model.ckpt.
    I1110 14:48:34.326113 22704 basic_session_run_hooks.py:606] Saving checkpoints for 6639 into tmp\model.ckpt.
    I1110 14:48:58.776925 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.450146
    I1110 14:50:49.779477 22704 basic_session_run_hooks.py:692] global_step/sec: 0.450146
    I1110 14:51:00.875863 22704 train.py:699] Saving summaries.
    I1110 14:53:02.742601 22704 train.py:699] Saving summaries.
    INFO:tensorflow:Saving checkpoints for 6775 into tmp\model.ckpt.
    I1110 14:53:35.923911 22704 basic_session_run_hooks.py:606] Saving checkpoints for 6775 into tmp\model.ckpt.
    INFO:tensorflow:global_step/sec: 0.451308
    I1110 14:54:31.357891 22704 basic_session_run_hooks.py:692] global_step/sec: 0.451308
    I1110 14:55:04.551207 22704 train.py:699] Saving summaries.
    I1110 14:57:04.588440 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.450366
    I1110 14:58:13.399682 22704 basic_session_run_hooks.py:692] global_step/sec: 0.450366
    INFO:tensorflow:Saving checkpoints for 6911 into tmp\model.ckpt.
    I1110 14:58:37.843343 22704 basic_session_run_hooks.py:606] Saving checkpoints for 6911 into tmp\model.ckpt.
    I1110 14:59:06.808158 22704 train.py:699] Saving summaries.
    I1110 15:01:08.908625 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.450214
    I1110 15:01:55.517238 22704 basic_session_run_hooks.py:692] global_step/sec: 0.450214
    I1110 15:03:10.921834 22704 train.py:699] Saving summaries.
    INFO:tensorflow:Saving checkpoints for 7047 into tmp\model.ckpt.
    I1110 15:03:39.707013 22704 basic_session_run_hooks.py:606] Saving checkpoints for 7047 into tmp\model.ckpt.
    I1110 15:05:12.979795 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.450701
    I1110 15:05:37.393572 22704 basic_session_run_hooks.py:692] global_step/sec: 0.450701
    I1110 15:07:14.825396 22704 train.py:699] Saving summaries.
    INFO:tensorflow:Saving checkpoints for 7183 into tmp\model.ckpt.
    I1110 15:08:41.291411 22704 basic_session_run_hooks.py:606] Saving checkpoints for 7183 into tmp\model.ckpt.
    I1110 15:09:16.842427 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.451103
    I1110 15:09:19.071444 22704 basic_session_run_hooks.py:692] global_step/sec: 0.451103
    I1110 15:11:18.943379 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.450104
    I1110 15:13:01.242074 22704 basic_session_run_hooks.py:692] global_step/sec: 0.450104
    I1110 15:13:19.011693 22704 train.py:699] Saving summaries.
    INFO:tensorflow:Saving checkpoints for 7319 into tmp\model.ckpt.
    I1110 15:13:43.461391 22704 basic_session_run_hooks.py:606] Saving checkpoints for 7319 into tmp\model.ckpt.
    I1110 15:15:19.026129 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.44957
    I1110 15:16:43.677086 22704 basic_session_run_hooks.py:692] global_step/sec: 0.44957
    I1110 15:17:19.467475 22704 train.py:699] Saving summaries.
    INFO:tensorflow:Saving checkpoints for 7454 into tmp\model.ckpt.
    I1110 15:18:44.107581 22704 basic_session_run_hooks.py:606] Saving checkpoints for 7454 into tmp\model.ckpt.
    I1110 15:19:19.814222 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.448581
    I1110 15:20:26.602098 22704 basic_session_run_hooks.py:692] global_step/sec: 0.448581
    I1110 15:21:19.954692 22704 train.py:699] Saving summaries.
    I1110 15:23:22.118618 22704 train.py:699] Saving summaries.
    INFO:tensorflow:Saving checkpoints for 7589 into tmp\model.ckpt.
    I1110 15:23:44.325301 22704 basic_session_run_hooks.py:606] Saving checkpoints for 7589 into tmp\model.ckpt.
    INFO:tensorflow:global_step/sec: 0.449491
    I1110 15:24:09.076154 22704 basic_session_run_hooks.py:692] global_step/sec: 0.449491
    I1110 15:25:22.513109 22704 train.py:699] Saving summaries.
    I1110 15:27:22.640475 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.449571
    I1110 15:27:51.510495 22704 basic_session_run_hooks.py:692] global_step/sec: 0.449571
    INFO:tensorflow:Saving checkpoints for 7724 into tmp\model.ckpt.
    I1110 15:28:44.859986 22704 basic_session_run_hooks.py:606] Saving checkpoints for 7724 into tmp\model.ckpt.
    I1110 15:29:22.779891 22704 train.py:699] Saving summaries.
    I1110 15:31:22.959095 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.449335
    I1110 15:31:34.061386 22704 basic_session_run_hooks.py:692] global_step/sec: 0.449335
    I1110 15:33:25.142914 22704 train.py:699] Saving summaries.
    INFO:tensorflow:Saving checkpoints for 7859 into tmp\model.ckpt.
    I1110 15:33:45.163441 22704 basic_session_run_hooks.py:606] Saving checkpoints for 7859 into tmp\model.ckpt.
    INFO:tensorflow:global_step/sec: 0.449849
    I1110 15:35:16.358006 22704 basic_session_run_hooks.py:692] global_step/sec: 0.449849
    I1110 15:35:25.227358 22704 train.py:699] Saving summaries.
    I1110 15:37:27.057925 22704 train.py:699] Saving summaries.
    INFO:tensorflow:Saving checkpoints for 7995 into tmp\model.ckpt.
    I1110 15:38:46.942628 22704 basic_session_run_hooks.py:606] Saving checkpoints for 7995 into tmp\model.ckpt.
    INFO:tensorflow:global_step/sec: 0.450847
    I1110 15:38:58.162617 22704 basic_session_run_hooks.py:692] global_step/sec: 0.450847
    I1110 15:39:29.291585 22704 train.py:699] Saving summaries.
    I1110 15:41:30.094023 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.447982
    I1110 15:42:41.385693 22704 basic_session_run_hooks.py:692] global_step/sec: 0.447982
    I1110 15:43:30.277153 22704 train.py:699] Saving summaries.
    INFO:tensorflow:Saving checkpoints for 8130 into tmp\model.ckpt.
    I1110 15:43:48.061679 22704 basic_session_run_hooks.py:606] Saving checkpoints for 8130 into tmp\model.ckpt.
    I1110 15:45:30.529277 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.44942
    I1110 15:46:23.894812 22704 basic_session_run_hooks.py:692] global_step/sec: 0.44942
    I1110 15:47:30.949940 22704 train.py:699] Saving summaries.
    INFO:tensorflow:Saving checkpoints for 8265 into tmp\model.ckpt.
    I1110 15:48:48.946709 22704 basic_session_run_hooks.py:606] Saving checkpoints for 8265 into tmp\model.ckpt.
    I1110 15:49:31.316579 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.448513
    I1110 15:50:06.853780 22704 basic_session_run_hooks.py:692] global_step/sec: 0.448513
    I1110 15:51:33.324863 22704 train.py:699] Saving summaries.
    I1110 15:53:35.347165 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.450879
    I1110 15:53:48.642664 22704 basic_session_run_hooks.py:692] global_step/sec: 0.450879
    INFO:tensorflow:Saving checkpoints for 8401 into tmp\model.ckpt.
    I1110 15:53:50.842784 22704 basic_session_run_hooks.py:606] Saving checkpoints for 8401 into tmp\model.ckpt.
    I1110 15:55:37.511007 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.448361
    I1110 15:57:31.678093 22704 basic_session_run_hooks.py:692] global_step/sec: 0.448361
    I1110 15:57:38.346303 22704 train.py:699] Saving summaries.
    INFO:tensorflow:Saving checkpoints for 8536 into tmp\model.ckpt.
    I1110 15:58:51.821071 22704 basic_session_run_hooks.py:606] Saving checkpoints for 8536 into tmp\model.ckpt.
    I1110 15:59:38.659231 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.449032
    I1110 16:01:14.378376 22704 basic_session_run_hooks.py:692] global_step/sec: 0.449032
    I1110 16:01:38.844218 22704 train.py:699] Saving summaries.
    I1110 16:03:40.909350 22704 train.py:699] Saving summaries.
    INFO:tensorflow:Saving checkpoints for 8671 into tmp\model.ckpt.
    I1110 16:03:51.994749 22704 basic_session_run_hooks.py:606] Saving checkpoints for 8671 into tmp\model.ckpt.
    INFO:tensorflow:global_step/sec: 0.450421
    I1110 16:04:56.392966 22704 basic_session_run_hooks.py:692] global_step/sec: 0.450421
    I1110 16:05:43.058265 22704 train.py:699] Saving summaries.
    I1110 16:07:45.246074 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.450008
    I1110 16:08:38.611440 22704 basic_session_run_hooks.py:692] global_step/sec: 0.450008
    INFO:tensorflow:Saving checkpoints for 8807 into tmp\model.ckpt.
    I1110 16:08:54.159922 22704 basic_session_run_hooks.py:606] Saving checkpoints for 8807 into tmp\model.ckpt.
    I1110 16:09:45.401132 22704 train.py:699] Saving summaries.
    I1110 16:11:45.681109 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.448941
    I1110 16:12:21.358957 22704 basic_session_run_hooks.py:692] global_step/sec: 0.448941
    I1110 16:13:45.940017 22704 train.py:699] Saving summaries.
    INFO:tensorflow:Saving checkpoints for 8942 into tmp\model.ckpt.
    I1110 16:13:54.857154 22704 basic_session_run_hooks.py:606] Saving checkpoints for 8942 into tmp\model.ckpt.
    I1110 16:15:46.244583 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.449133
    I1110 16:16:04.009152 22704 basic_session_run_hooks.py:692] global_step/sec: 0.449133
    I1110 16:17:46.792553 22704 train.py:699] Saving summaries.
    INFO:tensorflow:Saving checkpoints for 9077 into tmp\model.ckpt.
    I1110 16:18:56.028622 22704 basic_session_run_hooks.py:606] Saving checkpoints for 9077 into tmp\model.ckpt.
    INFO:tensorflow:global_step/sec: 0.447212
    I1110 16:19:47.616841 22704 basic_session_run_hooks.py:692] global_step/sec: 0.447212
    I1110 16:19:47.618953 22704 train.py:699] Saving summaries.
    I1110 16:21:47.711539 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.449712
    I1110 16:23:29.981562 22704 basic_session_run_hooks.py:692] global_step/sec: 0.449712
    I1110 16:23:47.788151 22704 train.py:699] Saving summaries.
    INFO:tensorflow:Saving checkpoints for 9212 into tmp\model.ckpt.
    I1110 16:23:56.709326 22704 basic_session_run_hooks.py:606] Saving checkpoints for 9212 into tmp\model.ckpt.
    I1110 16:25:48.529720 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.448239
    I1110 16:27:13.076912 22704 basic_session_run_hooks.py:692] global_step/sec: 0.448239
    I1110 16:27:48.805431 22704 train.py:699] Saving summaries.
    INFO:tensorflow:Saving checkpoints for 9347 into tmp\model.ckpt.
    I1110 16:28:58.029675 22704 basic_session_run_hooks.py:606] Saving checkpoints for 9347 into tmp\model.ckpt.
    I1110 16:29:49.410681 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.448064
    I1110 16:30:56.259154 22704 basic_session_run_hooks.py:692] global_step/sec: 0.448064
    I1110 16:31:51.805472 22704 train.py:699] Saving summaries.
    I1110 16:33:53.957277 22704 train.py:699] Saving summaries.
    INFO:tensorflow:Saving checkpoints for 9477 into tmp\model.ckpt.
    I1110 16:33:58.631745 22704 basic_session_run_hooks.py:606] Saving checkpoints for 9477 into tmp\model.ckpt.
    INFO:tensorflow:global_step/sec: 0.423592
    I1110 16:34:52.335222 22704 basic_session_run_hooks.py:692] global_step/sec: 0.423592
    I1110 16:35:56.740673 22704 train.py:699] Saving summaries.
    I1110 16:37:57.245175 22704 train.py:699] Saving summaries.
    INFO:tensorflow:Saving checkpoints for 9583 into tmp\model.ckpt.
    I1110 16:38:59.488322 22704 basic_session_run_hooks.py:606] Saving checkpoints for 9583 into tmp\model.ckpt.
    INFO:tensorflow:global_step/sec: 0.34327
    I1110 16:39:43.652295 22704 basic_session_run_hooks.py:692] global_step/sec: 0.34327
    I1110 16:39:58.932457 22704 train.py:699] Saving summaries.
    I1110 16:42:00.190226 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.407585
    I1110 16:43:48.999607 22704 basic_session_run_hooks.py:692] global_step/sec: 0.407585
    INFO:tensorflow:Saving checkpoints for 9705 into tmp\model.ckpt.
    I1110 16:44:00.608581 22704 basic_session_run_hooks.py:606] Saving checkpoints for 9705 into tmp\model.ckpt.
    I1110 16:44:00.691512 22704 train.py:699] Saving summaries.
    I1110 16:46:01.262396 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.429706
    I1110 16:47:41.715994 22704 basic_session_run_hooks.py:692] global_step/sec: 0.429706
    I1110 16:48:02.726863 22704 train.py:699] Saving summaries.
    INFO:tensorflow:Saving checkpoints for 9834 into tmp\model.ckpt.
    I1110 16:49:00.970298 22704 basic_session_run_hooks.py:606] Saving checkpoints for 9834 into tmp\model.ckpt.
    I1110 16:50:03.927318 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.43017
    I1110 16:51:34.182311 22704 basic_session_run_hooks.py:692] global_step/sec: 0.43017
    I1110 16:52:04.295792 22704 train.py:699] Saving summaries.
    INFO:tensorflow:Saving checkpoints for 9964 into tmp\model.ckpt.
    I1110 16:54:02.137135 22704 basic_session_run_hooks.py:606] Saving checkpoints for 9964 into tmp\model.ckpt.
    I1110 16:54:04.547746 22704 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 0.432799
    I1110 16:55:25.236198 22704 basic_session_run_hooks.py:692] global_step/sec: 0.432799
    INFO:tensorflow:Saving checkpoints for 10000 into tmp\model.ckpt.
    I1110 16:55:25.238219 22704 basic_session_run_hooks.py:606] Saving checkpoints for 10000 into tmp\model.ckpt.


Train the model with the sample data provided on Github

Script 4:

    python train.py \
        --train_coords third_party/neuroproof_examples/validation_sample/tf_record_file \
        --data_volumes validation1:third_party/neuroproof_examples/validation_sample/grayscale_maps.h5:raw \
        --label_volumes validation1:third_party/neuroproof_examples/validation_sample/groundtruth.h5:stack \
        --model_name convstack_3d.ConvStack3DFFNModel \
        --model_args "{\"depth\": 6, \"fov_size\": [11, 11, 11], \"deltas\": [4, 4, 4]}" \
        --image_mean 128 \
        --image_stddev 33

Inputs and outputs of Script 4 in cmd.exe (ignored the warnings):

    python train.py --train_coords third_party/neuroproof_examples/validation_sample/tf_record_file --data_volumes validation1:third_party/neuroproof_examples/validation_sample/grayscale_maps.h5:raw --label_volumes validation1:third_party/neuroproof_examples/validation_sample/groundtruth.h5:stack --model_name convstack_3d.ConvStack3DFFNModel --model_args "{\"depth\": 6, \"fov_size\": [11, 11, 11], \"deltas\": [4, 4, 4]}" --image_mean 128 --image_stddev 33

    INFO:tensorflow:global_step/sec: 4.52468
    I1108 20:30:19.527693  5536 basic_session_run_hooks.py:692] global_step/sec: 4.52468
    INFO:tensorflow:global_step/sec: 4.83335
    I1108 20:30:40.217261  5536 basic_session_run_hooks.py:692] global_step/sec: 4.83335
    INFO:tensorflow:global_step/sec: 4.70584
    I1108 20:31:01.468465  5536 basic_session_run_hooks.py:692] global_step/sec: 4.70584
    INFO:tensorflow:global_step/sec: 4.70473
    I1108 20:31:22.722663  5536 basic_session_run_hooks.py:692] global_step/sec: 4.70473
    INFO:tensorflow:global_step/sec: 4.86521
    I1108 20:31:43.276757  5536 basic_session_run_hooks.py:692] global_step/sec: 4.86521
    I1108 20:31:57.830861  5536 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 4.49628
    I1108 20:32:05.518363  5536 basic_session_run_hooks.py:692] global_step/sec: 4.49628
    INFO:tensorflow:global_step/sec: 4.69724
    I1108 20:32:26.806470  5536 basic_session_run_hooks.py:692] global_step/sec: 4.69724
    INFO:tensorflow:global_step/sec: 4.50053
    I1108 20:32:49.026089  5536 basic_session_run_hooks.py:692] global_step/sec: 4.50053
    INFO:tensorflow:global_step/sec: 4.69328
    I1108 20:33:10.333145  5536 basic_session_run_hooks.py:692] global_step/sec: 4.69328
    INFO:tensorflow:global_step/sec: 4.5249
    I1108 20:33:32.433091  5536 basic_session_run_hooks.py:692] global_step/sec: 4.5249
    INFO:tensorflow:global_step/sec: 4.70142
    I1108 20:33:53.703239  5536 basic_session_run_hooks.py:692] global_step/sec: 4.70142
    I1108 20:33:57.786327  5536 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 4.67686
    I1108 20:34:15.085096  5536 basic_session_run_hooks.py:692] global_step/sec: 4.67686
    INFO:tensorflow:global_step/sec: 4.60588
    I1108 20:34:36.796478  5536 basic_session_run_hooks.py:692] global_step/sec: 4.60588
    INFO:tensorflow:Saving checkpoints for 1390 into tmp\model.ckpt.
    I1108 20:34:57.519129  5536 basic_session_run_hooks.py:606] Saving checkpoints for 1390 into tmp\model.ckpt.
    INFO:tensorflow:global_step/sec: 4.37275
    I1108 20:34:59.665398  5536 basic_session_run_hooks.py:692] global_step/sec: 4.37275
    INFO:tensorflow:global_step/sec: 4.83136
    I1108 20:35:20.363501  5536 basic_session_run_hooks.py:692] global_step/sec: 4.83136
    INFO:tensorflow:global_step/sec: 4.92617
    I1108 20:35:40.663249  5536 basic_session_run_hooks.py:692] global_step/sec: 4.92617
    I1108 20:35:57.839403  5536 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 4.8992
    I1108 20:36:01.074762  5536 basic_session_run_hooks.py:692] global_step/sec: 4.8992
    INFO:tensorflow:global_step/sec: 4.92037
    I1108 20:36:21.399444  5536 basic_session_run_hooks.py:692] global_step/sec: 4.92037
    INFO:tensorflow:global_step/sec: 4.91289
    I1108 20:36:41.753054  5536 basic_session_run_hooks.py:692] global_step/sec: 4.91289
    INFO:tensorflow:global_step/sec: 4.94927
    I1108 20:37:01.958057  5536 basic_session_run_hooks.py:692] global_step/sec: 4.94927
    INFO:tensorflow:global_step/sec: 4.91744
    I1108 20:37:22.293822  5536 basic_session_run_hooks.py:692] global_step/sec: 4.91744
    INFO:tensorflow:global_step/sec: 4.93223
    I1108 20:37:42.568637  5536 basic_session_run_hooks.py:692] global_step/sec: 4.93223
    I1108 20:37:58.040290  5536 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 4.86663
    I1108 20:38:03.116723  5536 basic_session_run_hooks.py:692] global_step/sec: 4.86663
    INFO:tensorflow:global_step/sec: 3.90316
    I1108 20:38:28.736959  5536 basic_session_run_hooks.py:692] global_step/sec: 3.90316
    INFO:tensorflow:global_step/sec: 4.29153
    I1108 20:38:52.038691  5536 basic_session_run_hooks.py:692] global_step/sec: 4.29153
    INFO:tensorflow:global_step/sec: 3.24011
    I1108 20:39:22.901835  5536 basic_session_run_hooks.py:692] global_step/sec: 3.24011
    INFO:tensorflow:global_step/sec: 3.95451
    I1108 20:39:48.189429  5536 basic_session_run_hooks.py:692] global_step/sec: 3.95451
    INFO:tensorflow:Saving checkpoints for 2735 into tmp\model.ckpt.
    I1108 20:39:57.523483  5536 basic_session_run_hooks.py:606] Saving checkpoints for 2735 into tmp\model.ckpt.
    I1108 20:39:58.037111  5536 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 4.10175
    I1108 20:40:12.569274  5536 basic_session_run_hooks.py:692] global_step/sec: 4.10175
    INFO:tensorflow:global_step/sec: 4.27196
    I1108 20:40:35.977715  5536 basic_session_run_hooks.py:692] global_step/sec: 4.27196
    INFO:tensorflow:global_step/sec: 2.15387
    I1108 20:41:22.405808  5536 basic_session_run_hooks.py:692] global_step/sec: 2.15387
    I1108 20:41:58.384820  5536 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 1.98355
    I1108 20:42:12.820496  5536 basic_session_run_hooks.py:692] global_step/sec: 1.98355
    INFO:tensorflow:global_step/sec: 2.08266
    I1108 20:43:00.835926  5536 basic_session_run_hooks.py:692] global_step/sec: 2.08266
    INFO:tensorflow:global_step/sec: 2.35474
    I1108 20:43:43.303400  5536 basic_session_run_hooks.py:692] global_step/sec: 2.35474
    I1108 20:43:58.189617  5536 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 3.88794
    I1108 20:44:09.024947  5536 basic_session_run_hooks.py:692] global_step/sec: 3.88794
    INFO:tensorflow:global_step/sec: 4.8282
    I1108 20:44:29.735598  5536 basic_session_run_hooks.py:692] global_step/sec: 4.8282
    INFO:tensorflow:global_step/sec: 4.78215
    I1108 20:44:50.646713  5536 basic_session_run_hooks.py:692] global_step/sec: 4.78215
    INFO:tensorflow:Saving checkpoints for 3633 into tmp\model.ckpt.
    I1108 20:44:57.579187  5536 basic_session_run_hooks.py:606] Saving checkpoints for 3633 into tmp\model.ckpt.
    INFO:tensorflow:global_step/sec: 4.72667
    I1108 20:45:11.803273  5536 basic_session_run_hooks.py:692] global_step/sec: 4.72667
    INFO:tensorflow:global_step/sec: 4.83122
    I1108 20:45:32.501961  5536 basic_session_run_hooks.py:692] global_step/sec: 4.83122
    INFO:tensorflow:global_step/sec: 4.81449
    I1108 20:45:53.272577  5536 basic_session_run_hooks.py:692] global_step/sec: 4.81449
    I1108 20:45:58.275177  5536 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 4.83426
    I1108 20:46:13.958266  5536 basic_session_run_hooks.py:692] global_step/sec: 4.83426
    INFO:tensorflow:global_step/sec: 4.88085
    I1108 20:46:34.446510  5536 basic_session_run_hooks.py:692] global_step/sec: 4.88085
    INFO:tensorflow:global_step/sec: 4.8918
    I1108 20:46:54.888878  5536 basic_session_run_hooks.py:692] global_step/sec: 4.8918
    INFO:tensorflow:global_step/sec: 4.85532
    I1108 20:47:15.484835  5536 basic_session_run_hooks.py:692] global_step/sec: 4.85532
    INFO:tensorflow:global_step/sec: 4.86215
    I1108 20:47:36.051869  5536 basic_session_run_hooks.py:692] global_step/sec: 4.86215
    INFO:tensorflow:global_step/sec: 4.87539
    I1108 20:47:56.563053  5536 basic_session_run_hooks.py:692] global_step/sec: 4.87539
    I1108 20:47:58.284453  5536 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 4.85274
    I1108 20:48:17.169989  5536 basic_session_run_hooks.py:692] global_step/sec: 4.85274
    INFO:tensorflow:global_step/sec: 4.87539
    I1108 20:48:37.681166  5536 basic_session_run_hooks.py:692] global_step/sec: 4.87539
    INFO:tensorflow:global_step/sec: 4.77873
    I1108 20:48:58.607240  5536 basic_session_run_hooks.py:692] global_step/sec: 4.77873
    INFO:tensorflow:global_step/sec: 4.90784
    I1108 20:49:18.982788  5536 basic_session_run_hooks.py:692] global_step/sec: 4.90784
    INFO:tensorflow:global_step/sec: 4.92351
    I1108 20:49:39.293507  5536 basic_session_run_hooks.py:692] global_step/sec: 4.92351
    INFO:tensorflow:Saving checkpoints for 5091 into tmp\model.ckpt.
    I1108 20:49:57.745195  5536 basic_session_run_hooks.py:606] Saving checkpoints for 5091 into tmp\model.ckpt.
    I1108 20:49:58.470256  5536 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 4.88989
    I1108 20:49:59.743868  5536 basic_session_run_hooks.py:692] global_step/sec: 4.88989
    INFO:tensorflow:global_step/sec: 4.87042
    I1108 20:50:20.275981  5536 basic_session_run_hooks.py:692] global_step/sec: 4.87042
    INFO:tensorflow:global_step/sec: 4.91723
    I1108 20:50:40.613629  5536 basic_session_run_hooks.py:692] global_step/sec: 4.91723
    INFO:tensorflow:global_step/sec: 4.88965
    I1108 20:51:01.063976  5536 basic_session_run_hooks.py:692] global_step/sec: 4.88965
    INFO:tensorflow:global_step/sec: 4.87989
    I1108 20:51:21.556221  5536 basic_session_run_hooks.py:692] global_step/sec: 4.87989
    INFO:tensorflow:global_step/sec: 4.88703
    I1108 20:51:42.018532  5536 basic_session_run_hooks.py:692] global_step/sec: 4.88703
    I1108 20:51:58.507458  5536 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 4.86569
    I1108 20:52:02.570600  5536 basic_session_run_hooks.py:692] global_step/sec: 4.86569
    INFO:tensorflow:global_step/sec: 4.91073
    I1108 20:52:22.935175  5536 basic_session_run_hooks.py:692] global_step/sec: 4.91073
    INFO:tensorflow:global_step/sec: 4.9028
    I1108 20:52:43.330668  5536 basic_session_run_hooks.py:692] global_step/sec: 4.9028
    INFO:tensorflow:global_step/sec: 4.92037
    I1108 20:53:03.654353  5536 basic_session_run_hooks.py:692] global_step/sec: 4.92037
    INFO:tensorflow:global_step/sec: 4.90424
    I1108 20:53:24.044860  5536 basic_session_run_hooks.py:692] global_step/sec: 4.90424
    INFO:tensorflow:global_step/sec: 4.89276
    I1108 20:53:44.483238  5536 basic_session_run_hooks.py:692] global_step/sec: 4.89276
    I1108 20:53:58.655363  5536 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 4.40155
    I1108 20:54:07.203518  5536 basic_session_run_hooks.py:692] global_step/sec: 4.40155
    INFO:tensorflow:global_step/sec: 4.76035
    I1108 20:54:28.209381  5536 basic_session_run_hooks.py:692] global_step/sec: 4.76035
    INFO:tensorflow:Saving checkpoints for 6445 into tmp\model.ckpt.
    I1108 20:54:57.838782  5536 basic_session_run_hooks.py:606] Saving checkpoints for 6445 into tmp\model.ckpt.
    INFO:tensorflow:global_step/sec: 2.44354
    I1108 20:55:09.133628  5536 basic_session_run_hooks.py:692] global_step/sec: 2.44354
    INFO:tensorflow:global_step/sec: 4.87895
    I1108 20:55:29.629860  5536 basic_session_run_hooks.py:692] global_step/sec: 4.87895
    INFO:tensorflow:global_step/sec: 4.8638
    I1108 20:55:50.189906  5536 basic_session_run_hooks.py:692] global_step/sec: 4.8638
    I1108 20:55:58.783937  5536 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 4.86733
    I1108 20:56:10.735035  5536 basic_session_run_hooks.py:692] global_step/sec: 4.86733
    INFO:tensorflow:global_step/sec: 4.89514
    I1108 20:56:31.163439  5536 basic_session_run_hooks.py:692] global_step/sec: 4.89514
    INFO:tensorflow:global_step/sec: 4.74345
    I1108 20:56:52.245128  5536 basic_session_run_hooks.py:692] global_step/sec: 4.74345
    INFO:tensorflow:global_step/sec: 4.87208
    I1108 20:57:12.770246  5536 basic_session_run_hooks.py:692] global_step/sec: 4.87208
    INFO:tensorflow:global_step/sec: 4.92182
    I1108 20:57:33.087946  5536 basic_session_run_hooks.py:692] global_step/sec: 4.92182
    INFO:tensorflow:global_step/sec: 4.86427
    I1108 20:57:53.646010  5536 basic_session_run_hooks.py:692] global_step/sec: 4.86427
    I1108 20:57:58.897429  5536 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 4.38869
    I1108 20:58:16.431862  5536 basic_session_run_hooks.py:692] global_step/sec: 4.38869
    INFO:tensorflow:global_step/sec: 4.19414
    I1108 20:58:40.274652  5536 basic_session_run_hooks.py:692] global_step/sec: 4.19414
    INFO:tensorflow:global_step/sec: 4.23111
    I1108 20:59:03.909114  5536 basic_session_run_hooks.py:692] global_step/sec: 4.23111
    INFO:tensorflow:global_step/sec: 4.09577
    I1108 20:59:28.324556  5536 basic_session_run_hooks.py:692] global_step/sec: 4.09577
    INFO:tensorflow:global_step/sec: 4.11345
    I1108 20:59:52.635068  5536 basic_session_run_hooks.py:692] global_step/sec: 4.11345
    INFO:tensorflow:Saving checkpoints for 7823 into tmp\model.ckpt.
    I1108 20:59:57.927951  5536 basic_session_run_hooks.py:606] Saving checkpoints for 7823 into tmp\model.ckpt.
    I1108 20:59:59.102166  5536 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 4.06165
    I1108 21:00:17.255580  5536 basic_session_run_hooks.py:692] global_step/sec: 4.06165
    INFO:tensorflow:global_step/sec: 4.06642
    I1108 21:00:41.847251  5536 basic_session_run_hooks.py:692] global_step/sec: 4.06642
    INFO:tensorflow:global_step/sec: 4.39393
    I1108 21:01:04.606906  5536 basic_session_run_hooks.py:692] global_step/sec: 4.39393
    INFO:tensorflow:global_step/sec: 4.32191
    I1108 21:01:27.743839  5536 basic_session_run_hooks.py:692] global_step/sec: 4.32191
    INFO:tensorflow:global_step/sec: 4.26392
    I1108 21:01:51.196460  5536 basic_session_run_hooks.py:692] global_step/sec: 4.26392
    I1108 21:01:59.090730  5536 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 4.35357
    I1108 21:02:14.166090  5536 basic_session_run_hooks.py:692] global_step/sec: 4.35357
    INFO:tensorflow:global_step/sec: 4.35452
    I1108 21:02:37.130755  5536 basic_session_run_hooks.py:692] global_step/sec: 4.35452
    INFO:tensorflow:global_step/sec: 4.41197
    I1108 21:02:59.796383  5536 basic_session_run_hooks.py:692] global_step/sec: 4.41197
    INFO:tensorflow:global_step/sec: 4.31377
    I1108 21:03:22.977989  5536 basic_session_run_hooks.py:692] global_step/sec: 4.31377
    INFO:tensorflow:global_step/sec: 4.26794
    I1108 21:03:46.408495  5536 basic_session_run_hooks.py:692] global_step/sec: 4.26794
    I1108 21:03:59.225730  5536 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 4.4004
    I1108 21:04:09.133699  5536 basic_session_run_hooks.py:692] global_step/sec: 4.4004
    INFO:tensorflow:global_step/sec: 4.34144
    I1108 21:04:32.167528  5536 basic_session_run_hooks.py:692] global_step/sec: 4.34144
    INFO:tensorflow:global_step/sec: 4.37975
    I1108 21:04:54.999873  5536 basic_session_run_hooks.py:692] global_step/sec: 4.37975
    INFO:tensorflow:Saving checkpoints for 9113 into tmp\model.ckpt.
    I1108 21:04:58.126819  5536 basic_session_run_hooks.py:606] Saving checkpoints for 9113 into tmp\model.ckpt.
    INFO:tensorflow:global_step/sec: 4.33546
    I1108 21:05:18.065471  5536 basic_session_run_hooks.py:692] global_step/sec: 4.33546
    INFO:tensorflow:global_step/sec: 4.30089
    I1108 21:05:41.316493  5536 basic_session_run_hooks.py:692] global_step/sec: 4.30089
    I1108 21:05:59.457116  5536 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 4.23727
    I1108 21:06:04.916585  5536 basic_session_run_hooks.py:692] global_step/sec: 4.23727
    INFO:tensorflow:global_step/sec: 4.13796
    I1108 21:06:29.084078  5536 basic_session_run_hooks.py:692] global_step/sec: 4.13796
    INFO:tensorflow:global_step/sec: 4.15649
    I1108 21:06:53.141848  5536 basic_session_run_hooks.py:692] global_step/sec: 4.15649
    INFO:tensorflow:global_step/sec: 4.2838
    I1108 21:07:16.485589  5536 basic_session_run_hooks.py:692] global_step/sec: 4.2838
    INFO:tensorflow:global_step/sec: 4.08046
    I1108 21:07:40.992607  5536 basic_session_run_hooks.py:692] global_step/sec: 4.08046
    I1108 21:07:59.542409  5536 train.py:699] Saving summaries.
    INFO:tensorflow:global_step/sec: 4.12544
    I1108 21:08:05.232417  5536 basic_session_run_hooks.py:692] global_step/sec: 4.12544
    INFO:tensorflow:global_step/sec: 4.24469
    I1108 21:08:28.791283  5536 basic_session_run_hooks.py:692] global_step/sec: 4.24469
    INFO:tensorflow:Saving checkpoints for 10000 into tmp\model.ckpt.
    I1108 21:08:28.793278  5536 basic_session_run_hooks.py:606] Saving checkpoints for 10000 into tmp\model.ckpt.